In [145]:
import qubic.toolchain as _tc
import qubic.rpc_client as _rc
import qubitconfig.qchip as _qc
from distproc.hwconfig import FPGAConfig, load_channel_configs
from pygsti.models import modelconstruction as _mc
import numpy as np
import matplotlib.pyplot as plt
import chipcalibration.xr_rpe as _xrpe
from importlib import reload
import qubic.state_disc as _sd
import chipcalibration.config as _cfg
import qubic.job_manager_jpm as _jm
import chipcalibration.rabi_experiments as _rabi
import logging
import pygsti
import scipy
logging.basicConfig(level=0)

In [146]:
reload(_xrpe)
reload(_rabi)
reload(_jm)

<module 'qubic.job_manager_jpm' from '/home/jpmarceaux/PycharmProjects/software/qubic/job_manager_jpm.py'>

In [147]:
target_qid = 'Q2'
control_qid = 'Q3'

In [148]:
qchipname = 'X4Y2'
fpga_config, qchip, channel_config = _cfg.load_configs(qchipname)
chanmap = {'Q0':7,'Q2':1,'Q1':4,'Q3':3,'Q4':5,'Q5':6,'Q6':2}

In [149]:
runner = _rc.CircuitRunnerClient(ip='192.168.1.247')
jobman = _jm.JobManager(fpga_config, channel_config, runner, qchip)

In [150]:
def ZX(theta):
    return scipy.linalg.expm(-1j/2 * theta * pygsti.sigmazx)

In [151]:
# make the processor spec
register = [control_qid, target_qid] # these correspond to the actual qubit id's on the qubic device
gate_names = ['Gxpi2', 'Gzpi2', 'Gcr']

availability = {
    'Gxpi2': [(qid, ) for qid in register],
    'Gzpi2': [(qid, ) for qid in register],
    'Gcr': [(control_qid, target_qid)],
}

pspec = pygsti.processors.QubitProcessorSpec(num_qubits=len(register), gate_names=gate_names,
                                 availability=availability, qubit_labels=register,
                                 nonstd_gate_unitaries={'Gcr' : ZX(np.pi/2)})
target_model = model = _mc.create_explicit_model(pspec)

In [192]:
#Make a noisy model for simulation
noisy_model_0 = target_model.copy()
noisy_model_0.set_all_parameterizations('full')
delta = 0.1
noisy_model_0['Gcr',register[0],register[1]] = pygsti.unitary_to_pauligate(ZX(np.pi/2+delta))
noisy_model_0 = noisy_model_0.depolarize(0.1)

In [193]:
# make a pygsti simulation manager
sim_manager = _jm.SimulationManager(noisy_model_0)

In [194]:
xr_rpe = _xrpe.RPE_XR_Experiment(noisy_model_0, control_qid, target_qid, max_max_depth=9)

In [195]:
xr_rpe.run_and_report(sim_manager, 1000, qchip)

Last good generation: 6
Estimated phase: 1.6698065774626192


In [109]:
np.pi/2

1.5707963267948966

In [70]:
target_model.state_space

ExplicitStateSpace[Q3(4)*Q2(4)]

In [8]:
# GMM Rabi
gmm_rabi = _rabi.GMMRabi([target_qid, control_qid], 0.2, np.linspace(1.e-9, 50.e-9, 10),
                       channel_configs=channel_config)

In [9]:
gmm_rabi.run_and_report(jobman, 1000, qchip)

done resolving block
done z-resolving block
done resolving block
done z-resolving block
done scheduling
done resolving block
done z-resolving block
done resolving block
done z-resolving block
done scheduling
done resolving block
done z-resolving block
done resolving block
done z-resolving block
done scheduling
done resolving block
done z-resolving block
done resolving block
done z-resolving block
done scheduling
done resolving block
done z-resolving block
done resolving block
done z-resolving block
done scheduling
done resolving block
done z-resolving block
done resolving block
done z-resolving block
done scheduling
done resolving block
done z-resolving block
done resolving block
done z-resolving block
done scheduling
done resolving block
done z-resolving block
done resolving block
done z-resolving block
done scheduling
done resolving block
done z-resolving block
done resolving block
done z-resolving block
done scheduling
done resolving block
done z-resolving block
done resolving block

KeyboardInterrupt: 